## DNN深度神经网络
就是把原有的多层神经网络扩展到深度学习里面，加上了BP反馈，使得loss收敛直至不变，同时也有dropout

dropout是指随机用一定概率把一些节点失效，进行参与训练，防止数据陷入overfitting 局部最优解

In [1]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [2]:
import tensorflow as tf

In [3]:
# Parameter
learning_rate = 0.001
training_iters = 200000
batch_size = 128
display_step = 20

In [4]:
# Network Parameters
n_inputs = 784
n_classes = 10
dropout = 0.8

In [5]:
# tf Graph input 
x = tf.placeholder(tf.float32, [None, n_inputs])
y = tf.placeholder(tf.float32, [None, n_classes])
keep_prob = tf.placeholder(tf.float32)


In [6]:
def init_weights(shape):
    return tf.Variable(tf.random_normal(shape, stddev=0.01)) # stddev 标注差

def dnn(_X, _weights, _biases, _dropout):
    _X = tf.nn.dropout(_X, _dropout)
    
    d1 = tf.nn.relu(tf.nn.bias_add(tf.matmul(_X, _weights['wd1']), _biases['bd1']), name='d1')
    d2x = tf.nn.dropout(d1, _dropout)
    d2 = tf.nn.relu(tf.nn.bias_add(tf.matmul(d2x, _weights['wd2']), _biases['bd2']), name='d2')
    
    dout = tf.nn.dropout(d2,_dropout)
    out = tf.matmul(dout, _weights['out']) + _biases['out']
    return out

In [7]:
# store layers weights and bias
weights = {
    'wd1': tf.Variable(tf.random_normal([784, 600], stddev=0.01)),
    'wd2': tf.Variable(tf.random_normal([600, 480], stddev=0.01)),
    'out': tf.Variable(tf.random_normal([480, 10]))
}
biases = {
    'bd1': tf.Variable(tf.random_normal([600])),
    'bd2': tf.Variable(tf.random_normal([480])),
    'out': tf.Variable(tf.random_normal([10]))
}

In [8]:
# Construct model
pred = dnn(x, weights, biases, keep_prob)

In [9]:
# define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

In [10]:
# evaluate model
correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [11]:
# init 
init = tf.global_variables_initializer()

In [14]:
with tf.Session() as sess:
    sess.run(init)
    step = 1
    while step*batch_size < training_iters:
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        sess.run(optimizer, feed_dict={x: batch_xs, y:batch_ys, keep_prob: dropout})
        if step % display_step == 0:
            acc = sess.run(accuracy, feed_dict={x: batch_xs, y:batch_ys, keep_prob: 1.})
            loss = sess.run(cost, feed_dict={x: batch_xs, y:batch_ys, keep_prob: 1.})
            print("Iter " + str(step*batch_size) + ", Minibatch Loss= " + "{:.6f}".format(loss) + ", Training Accuracy= " + "{:.5f}".format(acc))
        step += 1
    print('Optimization Finished!')
    # Calculate accuracy for 256 mnist test images
    print("Testing Accuracy:", sess.run(accuracy, feed_dict={x: mnist.test.images[:256], y: mnist.test.labels[:256], keep_prob: 1.}))

Iter 2560, Minibatch Loss= 1.271280, Training Accuracy= 0.67969
Iter 5120, Minibatch Loss= 0.760037, Training Accuracy= 0.82812
Iter 7680, Minibatch Loss= 0.625286, Training Accuracy= 0.78906
Iter 10240, Minibatch Loss= 0.607161, Training Accuracy= 0.79688
Iter 12800, Minibatch Loss= 0.471471, Training Accuracy= 0.92188
Iter 15360, Minibatch Loss= 0.277094, Training Accuracy= 0.92188
Iter 17920, Minibatch Loss= 0.370746, Training Accuracy= 0.87500
Iter 20480, Minibatch Loss= 0.370081, Training Accuracy= 0.89844
Iter 23040, Minibatch Loss= 0.250388, Training Accuracy= 0.91406
Iter 25600, Minibatch Loss= 0.386681, Training Accuracy= 0.85938
Iter 28160, Minibatch Loss= 0.364527, Training Accuracy= 0.90625
Iter 30720, Minibatch Loss= 0.275465, Training Accuracy= 0.89844
Iter 33280, Minibatch Loss= 0.160494, Training Accuracy= 0.96875
Iter 35840, Minibatch Loss= 0.173311, Training Accuracy= 0.93750
Iter 38400, Minibatch Loss= 0.268104, Training Accuracy= 0.92188
Iter 40960, Minibatch Loss= 